＃ Model

送信データ →　変調器　→　伝送（雑音印加） →　復調器 →　受信データ

In [77]:
# import
import numpy as np
import sys

In [137]:
# データ生成器

def rndCode(codeSize, Type):
    """
    codeSize: 生成するランダム系列の大きさ（行数ｘ列数）を指定
    Type: タイプ指定 0: [0,1], 1: [-1, 1]
    """
    
    rndCode = np.random.randn(*codeSize)
    
    if rndCode.ndim > 2:
        sys.exit()
    
    dataSize = rndCode.shape
    if len(dataSize) == 1:
        sys.exit()
    
    if Type == 0:
        rndCode = np.where(rndCode <= 0, 0, 1)
    else:
        rndCode = np.where(rndCode <= 0, -1, 1)
    
    return rndCode

In [138]:
# BPSK変調器
def bpskMod(data):
    bpskSymbol = data.copy()
    np.place(bpskSymbol, bpskSymbol == 0, -1)
    
    return bpskSymbol

In [140]:
# 送信機

Ndata = 1024 # データ数
SNRdB = 10 # 雑音比

data = rndCode([Ndata, 1], 0)
bpskSymbol = bpskMod(data)

In [141]:
# ガウス雑音生成

def awgn(Pn, rn, cn):
    """
    Pn: 雑音電力
    ｒｎ: 行数
    cn: 列数
    """
    n = (np.random.randn(rn, cn)) + 1j * np.random.randn(rn, cn) * np.sqrt(Pn / 2)
    
    return n

In [159]:
# 雑音印加
NSymbol = len(bpskSymbol)
Pn = 10**(-SNRdB/10)
rSig = bpskSymbol + awgn(Pn, NSymbol, 1)

In [160]:
# BPSK復調器
def bpskDem(rSig):
    """
    rSig : 受信信号
    rData:　受信データ
    """
    rData = np.where(rSig < 0, 0, 1)
    
    return rData

In [163]:
# 受信機
rData = bpskDem(rSig)

In [165]:
# BER比較器
def ber(data1, data2):
    BER = np.sum(np.abs(data1-data2))/data1.size
    return BER

In [166]:
# ビットエラーレート（BER）の測定
ber(data, rData)

0.1650390625